In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('.')))
import librosa
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from utils.util import *
from functools import partial
from multiprocessing import Pool
from tqdm import tqdm
import pickle
import soundfile as sf

PATH_MIX = './mdx/B/mdx-net-submission/data/test_mackenzie/'
PATH = './mdx/B/mdx-net-submission/data/results_mackenzie/baseline/'
mackenzie_list = list(Path(PATH_MIX).glob('*/mixture.wav'))
mackenzie_vocal_list = list(Path(PATH).glob('*/vocals.wav'))
mackenzie_bass_list = list(Path(PATH).glob('*/bass.wav'))
mackenzie_drums_list = list(Path(PATH).glob('*/drums.wav'))
mackenzie_others_list = list(Path(PATH).glob('*/other.wav'))
SAMPLE_RATE = 16000
print(len(mackenzie_list), len(mackenzie_vocal_list)) # Mackenzie의 Mixture(원본) 음성은 145개, Vocal 추출 음성은 131개임을 확인.

In [ ]:
# # Write guitar wav
# num_write = 0
# for mackenzie_v, mackenzie_b, mackenzie_d, mackenzie_o in zip(tqdm(mackenzie_vocal_list), mackenzie_bass_list, mackenzie_drums_list, mackenzie_others_list):
#     assert str(mackenzie_v).split('/')[-2] == str(mackenzie_b).split('/')[-2] == str(mackenzie_d).split('/')[-2] == str(mackenzie_o).split('/')[-2]
#     # title = str(mackenzie_m).split('/')[-2]
#     # audio_m, sr = librosa.load(str(mackenzie_m), sr=SAMPLE_RATE, mono=True)
#     # audio_v, sr = librosa.load(str(mackenzie_v), sr=SAMPLE_RATE, mono=True)
#     audio_b, sr = librosa.load(str(mackenzie_b), sr=SAMPLE_RATE, mono=True)
#     audio_d, sr = librosa.load(str(mackenzie_d), sr=SAMPLE_RATE, mono=True)
#     audio_o, sr = librosa.load(str(mackenzie_o), sr=SAMPLE_RATE, mono=True)
#     # audio_g_m = audio_m - audio_v
#     audio_g_p = audio_b + audio_d + audio_o
#     save_guitar_path = str(mackenzie_v).replace('vocals.wav','guitar.wav')
#     sf.write(save_guitar_path, audio_g_p, samplerate=SAMPLE_RATE)
#     # sf.write(f'../save/{title}_mixture_minus_guitar.wav', audio_g_m, samplerate=16000)
#     # sf.write(f'../save/{title}_guitar_plus_others_plus_bass.wav', audio_g_p, samplerate=16000)

mackenzie_guitar_list = list(Path(PATH).glob('*/guitar.wav'))
print(len(mackenzie_guitar_list))

In [ ]:
REF = 0.1
threshold = 32000 # 2 sec
# with Pool(6) as p:
#     a = list(tqdm(p.imap(partial(obtain_start_end_of_mute, REF=REF, threshold=threshold), mackenzie_vocal_list),total=len(mackenzie_vocal_list)))
# with open(f'/save/mackenzie_vocal_start_end_of_mute_{REF}_{threshold}.txt', 'wb') as f:
#     pickle.dump(a, f)
# del a
with open(f'./save/mackenzie_vocal_start_end_of_mute_{REF}_{threshold}.txt', 'rb') as f:
    mackenzie_vocal_start_end_of_mute = pickle.load(f)
# with Pool(6) as p:
#     b = list(tqdm(p.imap(partial(obtain_start_end_of_mute, REF=REF, threshold=threshold), mackenzie_guitar_list),total=len(mackenzie_guitar_list)))
# with open(f'./save/mackenzie_guitar_start_end_of_mute_{REF}_{threshold}.txt', 'wb') as f:
#     pickle.dump(b, f)
# del b
with open(f'./save/mackenzie_guitar_start_end_of_mute_{REF}_{threshold}.txt', 'rb') as f:
    mackenzie_guitar_start_end_of_mute = pickle.load(f)

In [ ]:
total_mute_section = 0
total_available_section = 0 # section larger than 5s
total_available_length = 0 # section larger than 5s
for filepath, audio, aligned_retain, mute_start_end in mackenzie_vocal_start_end_of_mute:
    total_mute_section += len(mute_start_end)
    prev_mute_end = 0
    for mute_start, mute_end in mute_start_end:
        # print(mute_start//SAMPLE_RATE, mute_end//SAMPLE_RATE)
        if mute_start - prev_mute_end > 5.12*SAMPLE_RATE: # Mute가 아닌 기간이 5초가 넘는 경우?
            total_available_section+=1
            total_available_length+=mute_start-prev_mute_end
        prev_mute_end = mute_end
print(total_mute_section, total_available_section, total_available_length//SAMPLE_RATE/60/60)

현수님이 제일 좋아요

In [ ]:
def transpose(a,b):
    '''
    [[note1, note2, ...], [spec1, spec2, ...]] 꼴을
    [(note1, spec1), (note2, spec2), ...] 꼴로 변환해줌
    '''
    return list(map(tuple, zip(a,b)))
mackenzie_vocal_guitar = transpose(mackenzie_vocal_start_end_of_mute, mackenzie_guitar_start_end_of_mute)

In [ ]:
def segment_mackenzie(tuples):
    mackenzie_vocal_tuple, mackenzie_guitar_tuple = tuples
    filepath, audio, aligned_retain, mute_start_end = mackenzie_vocal_tuple 
    filepath_, audio_, aligned_retain_, mute_start_end_ = mackenzie_guitar_tuple 
    assert str(filepath).split('/')[-2] == str(filepath_).split('/')[-2]
    assert len(audio) == len(aligned_retain) == len(audio_) == len(aligned_retain_)
    filename = str(filepath).split('/')[-2]

    sum_retain = aligned_retain_ + aligned_retain
    state = 0 # 1 means sound present, 0 means silent
    start = 0
    end = 0
    available_segment_vocal = []
    available_segment_guitar = []
    for idx in range(len(audio)):
        if idx == len(audio)-1:
            if state == 1:
                end = idx
                if end - start >= 5.12*SAMPLE_RATE: # 5.12초로 변경?
                    available_segment_vocal.append(audio[start: end])
                    available_segment_guitar.append(audio_[start: end])
        if state == 0 and sum_retain[idx] == 0:
            state = 1
            start = idx
        elif state == 1 and sum_retain[idx] < 0:
            state = 0
            end = idx-1
            if end - start >= 5.12*SAMPLE_RATE: # 5.12초로 변경?
                available_segment_vocal.append(audio[start: end])
                available_segment_guitar.append(audio_[start: end])
    segment_num = 0
    vocal_segments = []
    guitar_segments = []
    for segment_vocal, segment_guitar in zip(available_segment_vocal, available_segment_guitar):
        assert segment_vocal.shape == segment_guitar.shape # 보컬과 기타의 shape이 같아야 하는 가정 설정문(assert)
        sf.write(f'./segments/mackenzie/vocal/{filename}_{segment_num}.wav', segment_vocal, samplerate=SAMPLE_RATE)
        sf.write(f'./segments/mackenzie/guitar/{filename}_{segment_num}.wav', segment_guitar, samplerate=SAMPLE_RATE)
        segment_num += 1
        vocal_segments.append(segment_vocal)
        guitar_segments.append(segment_guitar)
    return (vocal_segments, guitar_segments)
    
with Pool(6) as p:
    vocal_guitar = list(tqdm(p.imap(segment_mackenzie, mackenzie_vocal_guitar),total=len(mackenzie_vocal_guitar)))

In [ ]:
vocal_total = []
guitar_total = []
# vocal_wav = list(Path('./segments/mackenzie_norm/vocal').glob('*.wav'))
# guitar_wav = list(Path('./segments/mackenzie_norm/guitar').glob('*.wav'))
vocal_wav = list(Path('./segments/mackenzie/vocal').glob('*.wav'))
guitar_wav = list(Path('./segments/mackenzie/guitar').glob('*.wav'))
vocal_wav = sorted(vocal_wav)
guitar_wav = sorted(guitar_wav)

filename_list = []
for filename in vocal_wav:
    filename_ = '_'.join(str(filename).split('_')[:-1])
    filename_list.append(filename_)
filename_list = [*set(filename_list)]
filename_list= sorted(filename_list)
for i in filename_list:
    print(i)
filename_list_train = filename_list[:int(0.8*len(filename_list))+4]
filename_list_test = filename_list[int(0.8*len(filename_list))+4:]
segment_name_train = []
segment_name_test = []
for filename in vocal_wav:
    if '_'.join(str(filename).split('_')[:-1]) in filename_list_train:
        segment_name_train.append(filename)
    elif '_'.join(str(filename).split('_')[:-1]) in filename_list_test:
        segment_name_test.append(filename)
    else:
        raise ValueError

In [ ]:
with open('./feature/mackenzie_train_list_vocal.txt', 'w') as f:
    for vocal in tqdm(segment_name_train):
        f.write(str(vocal))
        f.write('\n')
with open('./feature/mackenzie_test_list_vocal.txt', 'w') as f:
    for vocal in tqdm(segment_name_test):
        f.write(str(vocal))
        f.write('\n')
# vocal : 716, test 179

In [ ]:
print(len(segment_name_train), len(segment_name_test), segment_name_train[:10])

In [ ]:
with open('./feature/mackenzie_train_list_vocal_norm.txt', 'r') as f:
    mackenzie_train_list_vocal = f.read().splitlines()
with open('./feature/mackenzie_test_list_vocal_norm.txt', 'r') as f:
    mackenzie_test_list_vocal = f.read().splitlines()
vocal_train = []
guitar_train = []
vocal_test = []
guitar_test = []
for train_vocal in tqdm(mackenzie_train_list_vocal):
    train_guitar = train_vocal.replace('/vocal/','/guitar/')
    wav_vocal, sr = sf.read(train_vocal)
    wav_guitar, sr = sf.read(train_guitar)
    wav_vocal = torch.tensor(wav_vocal, dtype=torch.float32)
    wav_guitar = torch.tensor(wav_guitar, dtype=torch.float32)
    vocal_train.append(wav_vocal)
    guitar_train.append(wav_guitar)
for test_vocal in tqdm(mackenzie_test_list_vocal):
    test_guitar = test_vocal.replace('/vocal/','/guitar/')
    wav_vocal, sr = sf.read(test_vocal)
    wav_guitar, sr = sf.read(test_guitar)
    wav_vocal = torch.tensor(wav_vocal, dtype=torch.float32)
    wav_guitar = torch.tensor(wav_guitar, dtype=torch.float32)
    vocal_test.append(wav_vocal)
    guitar_test.append(wav_guitar)

train_save_path = './feature/inpainting/mackenzie_unet_diffusion_inpainter_audio/train_norm.pt'
test_save_path = './feature/inpainting/mackenzie_unet_diffusion_inpainter_audio/test_norm.pt'
torch.save([vocal_train, guitar_train], train_save_path)
torch.save([vocal_test, guitar_test], test_save_path)

In [ ]:
# make train test splits in wav in groundtruth
import shutil
with open('./feature/mackenzie_train_list_vocal_norm.txt', 'r') as f:
    mackenzie_train_list_vocal = f.read().splitlines()
with open('./feature/mackenzie_test_list_vocal_norm.txt', 'r') as f:
    mackenzie_test_list_vocal = f.read().splitlines()
for train_vocal in tqdm(mackenzie_train_list_vocal):
    target_path = train_vocal.replace('segments/mackenzie_norm/vocal', 'save/groundtruth/mackenzie_norm/vocal/train')
    shutil.copy(train_vocal, target_path)
for test_vocal in tqdm(mackenzie_test_list_vocal):
    target_path = test_vocal.replace('segments/mackenzie_norm/vocal', 'save/groundtruth/mackenzie_norm/vocal/test')
    shutil.copy(test_vocal, target_path)
